In [1]:
import torch
import torch.nn.functional as F

In [15]:
import os
from torch.utils.cpp_extension import load
os.environ['MAX_JOBS'] = '16'
module = load(
    name='m',
    sources=['conv_1d.cu', 'conv_2d.cu', 'main.cpp'],
    verbose=True
)

Using /home/seb/.cache/torch_extensions/py312_cu121 as PyTorch extensions root...
The input conditions for extension module m have changed. Bumping to version 6 and re-building as m_v6...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/seb/.cache/torch_extensions/py312_cu121/m/build.ninja...
/home/seb/CUDA/cudaenv/lib/python3.12/site-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module m_v6...
Using envvar MAX_JOBS (16) as the number of workers...


[1/4] c++ -MMD -MF main.o.d -DTORCH_EXTENSION_NAME=m_v6 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/seb/CUDA/cudaenv/lib/python3.12/site-packages/torch/include -isystem /home/seb/CUDA/cudaenv/lib/python3.12/site-packages/torch/include/torch/csrc/api/include -isystem /home/seb/CUDA/cudaenv/lib/python3.12/site-packages/torch/include/TH -isystem /home/seb/CUDA/cudaenv/lib/python3.12/site-packages/torch/include/THC -isystem /usr/local/cuda-12.3/include -isystem /home/seb/miniconda3/include/python3.12 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17 -c /home/seb/CUDA/pmpp/image_conv/main.cpp -o main.o 
[2/4] /usr/local/cuda-12.3/bin/nvcc --generate-dependencies-with-compile --dependency-output conv_2d.cuda.o.d -DTORCH_EXTENSION_NAME=m_v6 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/seb/

Loading extension module m_v6...


In [67]:
# a = torch.tensor([1.0, 2.0, 3.0, 4.0, 5], dtype=torch.float32, device='cuda')
# mask = torch.tensor([0.0, 1.0, 0.0], dtype=torch.float32, device='cuda')
a = torch.rand(512, device='cuda', dtype=torch.float32)
mask_size = 5
mask = torch.rand(mask_size, device='cuda', dtype=torch.float32)


torch_res = F.conv1d(a.unsqueeze(0).unsqueeze(0), mask.unsqueeze(0).unsqueeze(0), padding=mask_size//2).flatten()
my_res = module.conv1d(a, mask).flatten()

print(torch.allclose(my_res, torch_res))
# my_res, torch_res

True


In [16]:
b = torch.rand((512, 512), device='cuda', dtype=torch.float32)
mask_size = (3, 3)
mask = torch.rand(mask_size, device='cuda', dtype=torch.float32)
# b = torch.ones((3, 3), device='cuda', dtype=torch.float32)
# mask = torch.ones((3, 3), device='cuda', dtype=torch.float32)
# mask = torch.tensor([[0, 1, 0], 
#                      [1, 0, 1], 
#                      [0, 1, 0]], device='cuda', dtype=torch.float32)

torch_res = F.conv2d(b.unsqueeze(0).unsqueeze(0), mask.unsqueeze(0).unsqueeze(0), padding=mask_size[0]//2)
my_res = module.conv2d(b, mask)
print(torch.allclose(torch_res, my_res))
torch_res, my_res

True


(tensor([[[[0.9701, 1.4021, 1.3976,  ..., 1.5615, 1.2028, 1.2654],
           [1.4884, 2.0791, 2.0830,  ..., 2.7148, 2.0361, 2.3480],
           [1.7177, 2.7809, 3.1708,  ..., 2.7130, 2.1817, 1.8365],
           ...,
           [0.8189, 2.3894, 2.2781,  ..., 2.6227, 2.3632, 1.3460],
           [1.2466, 2.4315, 2.3588,  ..., 2.4311, 2.1068, 1.1572],
           [1.4461, 1.6468, 1.4968,  ..., 2.0124, 1.4759, 0.6782]]]],
        device='cuda:0'),
 tensor([[0.9701, 1.4021, 1.3976,  ..., 1.5615, 1.2028, 1.2654],
         [1.4884, 2.0791, 2.0830,  ..., 2.7148, 2.0361, 2.3480],
         [1.7177, 2.7809, 3.1708,  ..., 2.7130, 2.1817, 1.8365],
         ...,
         [0.8189, 2.3894, 2.2781,  ..., 2.6227, 2.3632, 1.3460],
         [1.2466, 2.4315, 2.3588,  ..., 2.4311, 2.1068, 1.1572],
         [1.4461, 1.6468, 1.4968,  ..., 2.0124, 1.4759, 0.6782]],
        device='cuda:0'))